### Многоклассовая классификация для разделения абзацев резюме на категории

Категории:

* опыт работы
* образование
* навыки, умения
* общая информация о себе
* желаемая позиция
* зарплатные и другие ожидания 
* прочее

In [1]:
import os
import json
import random
import pandas as pd
import numpy as np
import nltk
import string
from joblib import dump, load
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk import sent_tokenize

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.metrics import accuracy_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
import warnings
warnings.simplefilter('ignore')

from config import CONFIG
from utils import convert_parsed_item_to_string, text_preprocessing

In [2]:
with open(f'{CONFIG.DATA_FOLDER}/rabota_cvs.json', 'r', encoding='utf-8') as f:
    cvs = json.load(f)
with open(f'{CONFIG.DATA_FOLDER}/hh_cvs.json', 'r', encoding='utf-8') as f:
    cvs.extend(json.load(f))

In [3]:
categories = ['position_name', 'education_list', 'skills', 'aboutme_info', 'salary', 'experience_list', 'other']
other_categories_rabota = ['city', 'citizenship', 'marriage_info', 'sex']
other_categories_hh = ['gender', 'age', 'city', 'spec_cat', 'education_degree']

df = pd.DataFrame()
arrays = []
for category in categories[:-1]:
    arrays.append([
        convert_parsed_item_to_string(cv[category]) 
        for cv in cvs if category in cv.keys() and cv[category]
    ])

other_arr = []
for cv in cvs:
    rabota_name = random.choice(other_categories_rabota)
    hh_name = random.choice(other_categories_hh)
    if rabota_name in cv.keys():
        other_arr.append(cv[rabota_name])
    elif hh_name in cv.keys():
        other_arr.append(cv[hh_name])
    
arrays.append(other_arr)

In [4]:
min_size = min(list(map(len, arrays)))
for category, array in zip(categories, arrays):
    df[category] = array[:min_size]
    
df.head()

,position_name,education_list,skills,aboutme_info,salary,experience_list,other
0,Программист,г.в.. Среднее. .,"Компьютеры, сотовая связь, Бытовая техника, эл...","Целеустремленный, ответственный, пунктуальный....",по договоренности,"ПАО ВЫМПЕЛКОМ. Москва. 2020, декабрь — продолж...","Холост , Детей нет"
1,"Продавец-консультант, менеджер по продажам, ра...",2018 г.в.. Среднее. .,"Css, html5, html-верстка, javascript, ajax, jq...",В этом году закончил университет и получил сте...,40 000 руб.,"Фриланс. Москва. 2019, февраль — 2020, июнь, 1...",Москва
2,frontend-разработчик,2023 г.в.. Неполное высшее. НИУ МЭИ. Информати...,"Интернет-сервисы, Заказное ПО, Банковское ПО, ...","Веду здоровый образ жизни, вредные привычки от...",20 000 руб.,"Инфокуб, IT-Stream. Москва. 2018, июль — 2020,...",Москва
3,"Программист Java, Javascript",2019 г.в.. Высшее образование (специалист). БН...,"C++, html5, javascript, python, json, css3, re...",Сайт-резюме: https://nightproger1.github.io/CV/,по договоренности,"СБЕР. Чита. 2020, сентябрь — продолжаю работат...",Беларусь
4,Программист,2019 г.в.. Высшее образование (специалист). Пн...,"Adobe photoshop, ms office, ms office excel, m...",Обладаю богатым опытом в реверс-инженерии (руч...,60 000 руб.,"ООО ВТБ МС. Москва. 2017, ноябрь — продолжаю р...",мужчина


In [5]:
for category in categories:
    df[category] = df[category].apply(text_preprocessing)
    
df.head()

,position_name,education_list,skills,aboutme_info,salary,experience_list,other
0,программист,гв среднее,компьютеры сотовая связь бытовая техника элект...,целеустремленный ответственный пунктуальный жи...,договоренности,пао вымпелком москва декабрь продолжаю работат...,холост детей
1,продавецконсультант менеджер продажам разработ...,гв среднее,css htmlверстка javascript ajax jquery git fle...,году закончил университет получил степень бака...,руб,фриланс москва февраль июнь год мес администра...,москва
2,frontendразработчик,гв неполное высшее ниу мэи информатика вычисли...,интернетсервисы заказное банковское разработка...,веду здоровый образ жизни вредные привычки отс...,руб,инфокуб itstream москва июль сентябрь года мес...,москва
3,программист java javascript,гв высшее образование специалист бнту беларусс...,c javascript python json react опп,сайтрезюме https,договоренности,сбер чита сентябрь продолжаю работать мес спец...,беларусь
4,программист,гв высшее образование специалист пнипу програм...,adobe photoshop ms office ms office excel ms o...,обладаю богатым опытом реверсинженерии ручном ...,руб,ооо втб мс москва ноябрь продолжаю работать го...,мужчина


### Кросс-валидация

In [6]:
X = []
y = []
for i, category in enumerate(categories):
    X += df[category].tolist()
    y += [i] * len(df[category]) 
    
X = np.array(X)
y = np.array(y)

In [9]:
classifiers = [GaussianNB(), MultinomialNB(), BernoulliNB(), LogisticRegression(multi_class='multinomial')]
metrics = [(accuracy_score, {}), (f1_score, {'average': 'macro'})]
skf = StratifiedKFold(n_splits=5)
cross_val_results = {
    classifier.__class__.__name__: {metric[0].__name__ : [] for metric in metrics}
    for classifier in classifiers
}

for train_idx, test_idx in skf.split(X, y):
    vectorizer = TfidfVectorizer(max_features=1000)
    vectorizer.fit(X[train_idx])
    X_train = vectorizer.transform(X[train_idx]).toarray()
    X_test = vectorizer.transform(X[test_idx]).toarray()
    y_train = y[train_idx]
    y_test = y[test_idx]
    for classifier in classifiers:
        classifier.fit(X_train, y_train)
        y_train_pred = classifier.predict(X_train)
        y_pred = classifier.predict(X_test)
        for metric in metrics:
            classifier_name = classifier.__class__.__name__
            metric_name = metric[0].__name__
            metric_params = metric[1]
            cross_val_results[classifier_name][metric_name].append(metric[0](y_test, y_pred, **metric_params))

In [10]:
mean_results = {
    classifier_name: {
        metric_name: np.mean(metric_arr) 
        for metric_name, metric_arr in item.items()
    } for classifier_name, item in cross_val_results.items()
}
df_score = pd.DataFrame(mean_results)
df_score.head()

,GaussianNB,MultinomialNB,BernoulliNB,LogisticRegression
accuracy_score,0.912821,0.918055,0.907338,0.946412
f1_score,0.912525,0.917689,0.907273,0.946600


### Обучение итоговой модели

In [11]:
vectorizer = TfidfVectorizer(max_features=1000)
classifier = LogisticRegression(multi_class='multinomial')
X_transformed = vectorizer.fit_transform(X).toarray()
classifier.fit(X_transformed, y)

LogisticRegression(multi_class='multinomial')

In [12]:
if not os.path.exists(CONFIG.MODELS_FOLDER):
    os.makedirs(CONFIG.MODELS_FOLDER)

dump(vectorizer, f'{CONFIG.MODELS_FOLDER}/{CONFIG.MULTI_VECTORIZER_NAME}') 
dump(classifier, f'{CONFIG.MODELS_FOLDER}/{CONFIG.MULTI_CLASSIFIER_NAME}') 

['models/multi_classifier.joblib']